In [63]:
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, \
Flatten, Conv2D, AveragePooling2D, MaxPooling2D, UpSampling2D, Dropout, GlobalAveragePooling2D

from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.initializers import glorot_uniform

In [2]:
def bottleneck_residual_block(X, f, filters, stage, block, reduce=False, s=2):
    
    # Defining the name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value to a new variable
    
    X_shortcut = X
    
    if reduce:
        X = Conv2D(filters = F1, kernel_size = (1,1), strides = (s,s), padding='valid', name= conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
        X = Activation('relu')(X)
        
        X_shortcut = Conv2D(filters = F3, kernel_size = (1,1), strides = (s,s), padding='valid', name = conv_name_base + '1',
                           kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
        X_shortcut = BatchNormalization(axis = 3, name=bn_name_base + '1')(X_shortcut)
        
    else:
        # first component of main
        X = Conv2D(filters = F1, kernel_size = (1,1), strides = (1,1), padding='valid', name= conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
        X = Activation('relu')(X)
        
    # Second Component of Main path
    X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), padding='valid', name=conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
    
    
    # Final step: Add shortcut value to main path, and pass it through the ReLu Activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [94]:
def ResNet50(input_shape, classes):
    
    X_input = Input(input_shape)
    
    X = UpSampling2D((2,2))(X_input)
    X = UpSampling2D((2,2))(X)
    X = UpSampling2D((2,2))(X)
    
    # Stage 1
    X = Conv2D(64, (7,7), strides=(2,2), name='conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides=(2,2))(X)
    
    # Stage 2
    X = bottleneck_residual_block(X, 3, [64, 64, 256], stage=2, block='a', reduce=True, s=1)
    X = bottleneck_residual_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = bottleneck_residual_block(X, 3, [64, 64, 256], stage=2, block='c')
    
    
    # Stage 3
    X = bottleneck_residual_block(X, 3, [128, 128, 512], stage=3, block='a', reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [128,128, 512], stage=3, block='b')
    X = bottleneck_residual_block(X, 3, [128,128, 512], stage=3, block='c')
    X = bottleneck_residual_block(X, 3, [128,128, 512], stage=3, block='d')
    
    # Stage 4
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='a', reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = bottleneck_residual_block(X, 3, [256, 256, 1024], stage=4, block='f')
    
    # Stage 5
    X = bottleneck_residual_block(X, 3, [512, 512, 2048], stage=5, block='a', reduce=True, s=2)
    X = bottleneck_residual_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = bottleneck_residual_block(X, 3, [512, 512, 2048], stage=5, block='c')
    
    # AVGPOOL
    X = GlobalAveragePooling2D(name='Avg_pool')(X)
    
    # Output Layer
    X = Flatten()(X)
    X = Dense(200, activation='relu')(X)
    X = Dropout(0.25)(X)
    X = BatchNormalization()(X)
    X = Dense(500, activation='relu')(X)
    X = Dense(classes, activation = 'softmax', name = 'fc' + str(classes), kernel_initializer= glorot_uniform(seed=0))(X)
    
    # Create The Model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    
    return model

In [95]:
resnet50 = ResNet50(input_shape = (32,32,3), classes=100)

In [96]:
resnet50.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 up_sampling2d_21 (UpSampling2D  (None, 64, 64, 3)   0           ['input_9[0][0]']                
 )                                                                                                
                                                                                                  
 up_sampling2d_22 (UpSampling2D  (None, 128, 128, 3)  0          ['up_sampling2d_21[0][0]']       
 )                                                                                                
                                                                                           

 activation_399 (Activation)    (None, 62, 62, 64)   0           ['bn2c_branch2a[0][0]']          
                                                                                                  
 res2c_branch2b (Conv2D)        (None, 62, 62, 64)   36928       ['activation_399[0][0]']         
                                                                                                  
 bn2c_branch2b (BatchNormalizat  (None, 62, 62, 64)  256         ['res2c_branch2b[0][0]']         
 ion)                                                                                             
                                                                                                  
 activation_400 (Activation)    (None, 62, 62, 64)   0           ['bn2c_branch2b[0][0]']          
                                                                                                  
 res2c_branch2c (Conv2D)        (None, 62, 62, 256)  16640       ['activation_400[0][0]']         
          

                                                                                                  
 activation_409 (Activation)    (None, 31, 31, 128)  0           ['bn3c_branch2b[0][0]']          
                                                                                                  
 res3c_branch2c (Conv2D)        (None, 31, 31, 512)  66048       ['activation_409[0][0]']         
                                                                                                  
 bn3c_branch2c (BatchNormalizat  (None, 31, 31, 512)  2048       ['res3c_branch2c[0][0]']         
 ion)                                                                                             
                                                                                                  
 add_133 (Add)                  (None, 31, 31, 512)  0           ['bn3c_branch2c[0][0]',          
                                                                  'activation_407[0][0]']         
          

 res4b_branch2c (Conv2D)        (None, 16, 16, 1024  263168      ['activation_418[0][0]']         
                                )                                                                 
                                                                                                  
 bn4b_branch2c (BatchNormalizat  (None, 16, 16, 1024  4096       ['res4b_branch2c[0][0]']         
 ion)                           )                                                                 
                                                                                                  
 add_136 (Add)                  (None, 16, 16, 1024  0           ['bn4b_branch2c[0][0]',          
                                )                                 'activation_416[0][0]']         
                                                                                                  
 activation_419 (Activation)    (None, 16, 16, 1024  0           ['add_136[0][0]']                
          

                                                                                                  
 add_139 (Add)                  (None, 16, 16, 1024  0           ['bn4e_branch2c[0][0]',          
                                )                                 'activation_425[0][0]']         
                                                                                                  
 activation_428 (Activation)    (None, 16, 16, 1024  0           ['add_139[0][0]']                
                                )                                                                 
                                                                                                  
 res4f_branch2a (Conv2D)        (None, 16, 16, 256)  262400      ['activation_428[0][0]']         
                                                                                                  
 bn4f_branch2a (BatchNormalizat  (None, 16, 16, 256)  1024       ['res4f_branch2a[0][0]']         
 ion)     

                                                                                                  
 activation_437 (Activation)    (None, 8, 8, 2048)   0           ['add_142[0][0]']                
                                                                                                  
 res5c_branch2a (Conv2D)        (None, 8, 8, 512)    1049088     ['activation_437[0][0]']         
                                                                                                  
 bn5c_branch2a (BatchNormalizat  (None, 8, 8, 512)   2048        ['res5c_branch2a[0][0]']         
 ion)                                                                                             
                                                                                                  
 activation_438 (Activation)    (None, 8, 8, 512)    0           ['bn5c_branch2a[0][0]']          
                                                                                                  
 res5c_bra

In [97]:
# from tensorflow.keras.utils import plot_model

# plot_model(model, to_file="images/resnet50.png", show_shapes=True)

In [98]:
from tensorflow.keras.datasets import cifar100
#from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.utils import np_utils

In [99]:
num_classes = 100
nb_epochs = 1

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

In [100]:
x_train.shape

(50000, 32, 32, 3)

In [101]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [102]:
# model = Sequential()
# model.add(UpSampling2D((2,2)))
# model.add(UpSampling2D((2,2)))
# model.add(UpSampling2D((2,2)))
# model.add(resnet50)

In [103]:
#model.build(input_shape  = (32,32,3)) 

In [104]:
#model.summary()

In [105]:
resnet50.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [107]:
historytemp = resnet50.fit(x_train[:1000], 
                        y_train[:1000],
                        epochs=10,
                        steps_per_epoch=32,
                        batch_size = 32)#,
                        #validation_data=(x_test, y_test))

Epoch 1/10
32/32 [==============================] - 376s 12s/step - loss: 4.6736 - accuracy: 0.0290
Epoch 2/10
32/32 [==============================] - 371s 12s/step - loss: 4.5193 - accuracy: 0.0250
Epoch 3/10
32/32 [==============================] - 332s 10s/step - loss: 4.3955 - accuracy: 0.0420
Epoch 4/10
16/32 [==============>...............] - ETA: 2:44 - loss: 4.3052 - accuracy: 0.0254

KeyboardInterrupt: 

1562.5